In [ ]:
pip install --upgrade "langchain>=0.1.0" "langchain-community>=0.0.29" "langchain-openai>=0.0.5" "openai>=1.12.0" "chromadb>=0.4.22" "langchain-chroma>=0.0.1" "python-dotenv>=1.0.0"

In [ ]:
# 벡터 임베딩
# Chroma 사용

import csv
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

open_ai_api_key = os.getenv('OPENAI_API_KEY')
course_csv_path = os.getenv('COURSE_CSV_PATH')

embedding_model = OpenAIEmbeddings(
    model = "text-embedding-3-large",
    openai_api_key = open_ai_api_key,
    request_timeout = 3000,
)

raw_documents = []
with open(course_csv_path,'r', encoding='utf-8-sig') as fh:
    reader = csv.DictReader(fh)
    for row in reader:
        course_text = f"{row['Course_Name']}: {row['Course_Description']}, {row['Course_Teacher']}"
        document = Document(
            page_content= course_text,
            metadata= {
                'course_id': row['Course_Id']
            },
            id=row['Course_Id']
        )
        raw_documents.append(document)

vector = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding_model, 
    persist_directory='./course_vector_db', 
    collection_name="courses"
)

In [ ]:
# Chroma DB에 접근하는 코드
from langchain_chroma import Chroma
from chromadb import Client, Settings

# 클라이언트 생성
client = Client(Settings(
    persist_directory='./course_vector_db',
    is_persistent=True
))

# 컬렉션 가져오기
collection = client.get_collection(name="courses")

# 컬렉션의 모든 항목 가져오기
results = collection.get(include=['embeddings', 'documents', 'metadatas'])

# 결과 확인
if results:
    print(f"저장된 문서 수: {len(results['ids'])}")
    # 처음 몇 개의 문서 출력
    for i in range(min(3, len(results['ids']))):
        print(f"\n문서 ID: {results['ids'][i]}")
        print(f"문서 내용: {results['documents'][i]}")
        print(f"메타데이터: {results['metadatas'][i]}")
        print(f"임베딩: {results['embeddings'][i].tolist()}")
else:
    print("컬렉션이 비어있거나 접근할 수 없습니다.")

In [ ]:
# 또는 LangChain의 Chroma를 사용할 경우:
db = Chroma(
    persist_directory='./course_vector_db',
    embedding_function=embedding_model,
    collection_name="courses"
)



# 간단한 검색 테스트
print(f"시맨틱 검색")
search_query = "데이터 분석 강의 추천해줘"
#search_query = "Recommend me a data analysis course"
print(f"검색 쿼리: {search_query}")
results = db.similarity_search_with_relevance_scores(search_query, k=5)
for doc, score in results:
    print(f"유사도 점수: {score:.4f}")  # 소수점 4자리까지 표시
    print(f"강좌명: {doc.page_content}")
    print(f"강좌 ID: {doc.metadata['course_id']}")
    print("-" * 50)

In [ ]:
pip install pymongo numpy

In [ ]:
from pymongo import MongoClient
import numpy as np
from chromadb import Client, Settings
from datetime import datetime
import os

# .env 파일 로드
load_dotenv()

mongo_url = os.getenv('MONGO_URL')
database_name = os.getenv('DATABASE_NAME')

# MongoDB 연결 설정
client = MongoClient(mongo_url)
db = client[database_name]  # 데이터베이스 선택
collection = db['courses']  # 컬렉션 선택

# Chroma DB 클라이언트 생성
chroma_client = Client(Settings(
    persist_directory='./course_vector_db',
    is_persistent=True
))

# Chroma 컬렉션 가져오기
chroma_collection = chroma_client.get_collection(name="courses")

# 모든 데이터 가져오기
results = chroma_collection.get(include=['embeddings', 'documents', 'metadatas'])

# MongoDB에 데이터 저장
for i in range(len(results['ids'])):
    document = {
        'id': results['ids'][i],
        'content': results['documents'][i],
        'metadata': results['metadatas'][i],
        'embeddings': results['embeddings'][i].tolist(),  # numpy array를 list로 변환
        'created_at': datetime.now()
    }
    print(document)
    
    # upsert=True로 설정하여 이미 존재하는 경우 업데이트
    collection.update_one(
        {'course_id': results['ids'][i]},
        {'$set': document},
        upsert=True
    )

print(f"Successfully stored {len(results['ids'])} documents in MongoDB")

# MongoDB에서 데이터 검색 예시
def find_course_by_id(course_id):
    return collection.find_one({'course_id': course_id})

# 사용 예시
if __name__ == "__main__":
    # 예시로 첫 번째 과정 검색
    sample_id = results['ids'][0]
    found_course = find_course_by_id(sample_id)
    if found_course:
        print(f"\nFound course: {found_course['content']}")
    else:
        print(f"Course not found: {found_course}")

In [ ]:
import openai
import os

# .env 파일 로드
load_dotenv()

openai.api_key  = os.getenv('OPENAI_API_KEY')

query_text = "데이터 분석 강의 추천해줘"

response = openai.embeddings.create(
    model="text-embedding-3-large",
    input=query_text
)

# 📊 queryVector 추출
query_vector = response.data[0].embedding

# 결과 출력
print("queryVector:", query_vector)
print("벡터 길이:", len(query_vector))